In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import os, glob, math, random
import ast
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import torch
import pydicom
from scipy.ndimage import zoom
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
df = pd.read_csv('/kaggle/input/rsna2025-extra/train_add_metadata_v5.csv')
df.loc[76, 'sorted_files']

"['/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10215833141558976135001043369327881438/1.2.826.0.1.3680043.8.498.59853755461586114842201654202370129432.dcm', '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10215833141558976135001043369327881438/1.2.826.0.1.3680043.8.498.86426531889402049965746231574415686240.dcm', '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10215833141558976135001043369327881438/1.2.826.0.1.3680043.8.498.12692137107562892693791410789847634864.dcm', '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10215833141558976135001043369327881438/1.2.826.0.1.3680043.8.498.26915188169562062070273927735870919164.dcm', '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10215833141558976135001043369327881438/1.2.826.0.1.3680043.8.498.10510826664699626647618046073389107865.dcm', '/kaggle/input/rsna-int

In [3]:
# ========== 小物ユーティリティ ==========
def _iou_xyxy(a, b):
    # a,b: [x1,y1,x2,y2]
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = max(0.0, (a[2]-a[0]) * (a[3]-a[1]))
    area_b = max(0.0, (b[2]-b[0]) * (b[3]-b[1]))
    union = area_a + area_b - inter + 1e-9
    return inter / union

def _ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

# ========== 簡易 Weighted Boxes Fusion（クラス別） ==========
def weighted_boxes_fusion(
    boxes, scores, labels,
    iou_thr=0.55,
    conf_type="avg"   # "avg" か "one_minus_prod"
):
    """
    複数モデルの出力をマージ。boxes/scores/labels はすべて結合済み1配列（同一画像）。
    - boxes: (N,4) xyxy
    - scores: (N,)
    - labels: (N,)
    返り値: fused_boxes, fused_scores, fused_labels
    """
    if len(boxes) == 0:
        return boxes, scores, labels

    order = np.argsort(-scores)
    boxes = boxes[order]; scores = scores[order]; labels = labels[order]

    clusters = []  # list of dict: {'box': np.array(4), 'score_sum': float, 'scores': [..], 'label': int, 'weight_sum': float, 'members': int}
    for b, s, c in zip(boxes, scores, labels):
        matched = False
        for cl in clusters:
            if cl['label'] != c:
                continue
            if _iou_xyxy(b, cl['box']) >= iou_thr:
                # 既存クラスタにマージ（座標はスコア重みで移動平均）
                w_old = cl['weight_sum']; w_new = s
                cl['box'] = (cl['box'] * w_old + b * w_new) / (w_old + w_new + 1e-9)
                cl['weight_sum'] += w_new
                cl['scores'].append(s)
                cl['members'] += 1
                matched = True
                break
        if not matched:
            clusters.append({
                'box': b.copy(),
                'label': int(c),
                'weight_sum': float(s),
                'scores': [float(s)],
                'members': 1,
            })

    fused_boxes, fused_scores, fused_labels = [], [], []
    for cl in clusters:
        fused_boxes.append(cl['box'])
        if conf_type == "one_minus_prod":
            # 1 - ∏(1-s) で統合信頼度
            p = 1.0
            for s in cl['scores']:
                p *= (1.0 - s)
            sc = 1.0 - p
        else:
            sc = float(np.mean(cl['scores']))
        fused_scores.append(sc)
        fused_labels.append(cl['label'])

    return np.vstack(fused_boxes), np.array(fused_scores), np.array(fused_labels, dtype=int)

# ========== 1画像に対する fold アンサンブル推論 ==========
def predict_ensemble_single_image(
    models,
    image,
    conf=0.2,
    iou=0.7,
    imgsz=320,
    per_model_max_det=1,      # 各モデルが返す最大検出数
    ensemble_iou=0.55,          # WBF のクラスタ閾値
    only_one=True,             # True で最終1件に絞る
    agnostic=False,             # True でクラス無視の後段NMSをしたい場合（今回はWBFなので通常Falseのまま推奨）
):
    """
    各 fold の best.pt を読み、単一画像に対して推論→WBFで統合。
    戻り値: dict {'boxes': (K,4), 'scores': (K,), 'labels': (K,)}
    """
    # 遅延ロードを避けたい場合は、外で [YOLO(p) for p in model_paths] を渡す実装に変えてもOK
    boxes_all = []
    scores_all = []
    labels_all = []

    for model in models:
        res = model.predict(
            source=image,
            conf=conf, iou=iou, imgsz=imgsz, max_det=per_model_max_det, verbose=False
        )[0]
        if res.boxes is None or len(res.boxes) == 0:
            continue
        xyxy = res.boxes.xyxy.cpu().numpy()
        sco  = res.boxes.conf.cpu().numpy()
        lab  = (res.boxes.cls.cpu().numpy().astype(int)
                if res.boxes.cls is not None else np.zeros(len(xyxy), dtype=int))
        boxes_all.append(xyxy); scores_all.append(sco); labels_all.append(lab)

    if len(boxes_all) == 0:
        return {'boxes': np.zeros((0,4), dtype=np.float32),
                'scores': np.zeros((0,), dtype=np.float32),
                'labels': np.zeros((0,), dtype=np.int32)}

    boxes = np.vstack(boxes_all)
    scores = np.hstack(scores_all)
    labels = np.hstack(labels_all)

    # 簡易 WBF
    f_boxes, f_scores, f_labels = weighted_boxes_fusion(
        boxes, scores, labels, iou_thr=ensemble_iou, conf_type="one_minus_prod"
    )

    # 1件に絞る（スコア最大）
    if only_one and len(f_scores) > 0:
        idx = int(np.argmax(f_scores))
        f_boxes = f_boxes[idx:idx+1]
        f_scores = f_scores[idx:idx+1]
        f_labels = f_labels[idx:idx+1]

    return {'boxes': f_boxes, 'scores': f_scores, 'labels': f_labels}

# ========== ディレクトリ一括推論 & オーバーレイ保存 & CSV ==========
def ensemble_infer_dir(
    models,
    image_paths,
    out_dir="/kaggle/working/ens_out",
    conf=0.2,
    iou=0.7,
    imgsz=320,
    per_model_max_det=1,
    ensemble_iou=0.55,
    only_one=True,
    draw_score=True,
):
    """
    images_root 配下の *.png / *.jpg を再帰探索して、fold アンサンブル推論。
    - オーバーレイ画像を out_dir/overlays 下に保存（元のサブフォルダ構造を維持）
    - 予測座標を CSV に保存（out_dir/ensemble_predictions.csv）
    """
    out_dir = Path(out_dir)
    overlays_dir = out_dir / "overlays"
    _ensure_dir(overlays_dir)

    
    rows = []
    # フォント（環境によっては使えないことがあるので None でフォールバック）
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", size=14)
    except:
        font = None

    for img_path in tqdm(image_paths, total=len(image_paths)):
        pred = predict_ensemble_single_image(
            models, str(img_path),
            conf=conf, iou=iou, imgsz=imgsz,
            per_model_max_det=per_model_max_det,
            ensemble_iou=ensemble_iou,
            only_one=only_one,
        )

        # CSV 行を作成
        if len(pred['scores']) == 0:
            rows.append({
                "image_path": str(img_path),
                "x1": np.nan, "y1": np.nan, "x2": np.nan, "y2": np.nan,
                "score": np.nan, "class": np.nan,
                "cx": np.nan, "cy": np.nan, "w": np.nan, "h": np.nan
            })
        else:
            for (x1,y1,x2,y2), sc, cls in zip(pred['boxes'], pred['scores'], pred['labels']):
                w = x2 - x1; h = y2 - y1; cx = x1 + w/2; cy = y1 + h/2
                rows.append({
                    "image_path": str(img_path),
                    "x1": float(x1), "y1": float(y1), "x2": float(x2), "y2": float(y2),
                    "score": float(sc), "class": int(cls),
                    "cx": float(cx), "cy": float(cy), "w": float(w), "h": float(h)
                })

        # オーバーレイ保存
        img = Image.open(img_path).convert("RGB")
        draw = ImageDraw.Draw(img)
        for (x1,y1,x2,y2), sc, cls in zip(pred['boxes'], pred['scores'], pred['labels']):
            draw.rectangle([x1,y1,x2,y2], outline=(255,0,0), width=2)
            if draw_score:
                txt = f"{int(cls)}:{sc:.2f}"
                if hasattr(draw, "textbbox"):
                    l, t, r, b = draw.textbbox((0, 0), txt, font=font)
                    tw, th = (r - l), (b - t)
                else:
                    # 古いPillow向けフォールバック
                    tw, th = draw.textsize(txt, font=font)
                #tw, th = draw.textlength(txt, font=font) if hasattr(draw, "textlength") else (len(txt)*8, 12)
                draw.rectangle([x1, max(0,y1-th-2), x1+tw+4, y1], fill=(255,0,0))
                draw.text((x1+2, max(0,y1-th-2)), txt, fill=(255,255,255), font=font)

        parts = img_path.parts
        i = parts.index("series")
        rel = Path(*parts[i+1:])
        out_img_path = overlays_dir / rel
        _ensure_dir(out_img_path.parent)
        img.save(out_img_path)

    # CSV 保存
    df = pd.DataFrame(rows)
    csv_path = out_dir / "ensemble_predictions.csv"
    _ensure_dir(csv_path.parent)
    df.to_csv(csv_path, index=False)
    print(f"Saved overlays to: {overlays_dir}")
    print(f"Saved CSV to:      {csv_path}")
    return {"csv": str(csv_path), "overlays_dir": str(overlays_dir), "df": df}


In [4]:
def pct_normalize(img, p_min, p_max):
    vmin = np.percentile(img, p_min)
    vmax = np.percentile(img, p_max)
    img = np.clip(img, vmin, vmax)
    img = (img - vmin) / (vmax - vmin + 1e-6)
    img = (img*255).astype(np.uint8)
    return img

def value_normalize(img, vmin, vmax):
    img = np.clip(img, vmin, vmax)
    img = (img - vmin) / (vmax - vmin + 1e-6)
    img = (img*255).astype(np.uint8)
    return img
    
def resample_zyx(arr: np.ndarray,
                 spacing: tuple[float, float, float],
                 new_spacing: tuple[float, float, float],
                 order: int) -> np.ndarray:
    """
    (Z,Y,X) を spacing -> new_spacing へリサンプリング。
    order=1: 画像（線形）, order=0: マスク（最近傍）
    """
    sz, sy, sx = spacing
    nsz, nsy, nsx = new_spacing
    zz, zy, zx = sz / nsz, sy / nsy, sx / nsx
    mode = "nearest" if order == 0 else "constant"
    return zoom(arr, zoom=(zz, zy, zx), order=order, mode=mode)
    
def load_dicom_array_2d(paths):
    imgs = []
    for path in paths:
        dcm = pydicom.dcmread(path)
        imgs.append(dcm.pixel_array)
    imgs = np.array(imgs)
    return imgs

def load_dicom_array_3d(path):
    dcm = pydicom.dcmread(path)
    imgs = dcm.pixel_array
    return imgs

In [5]:
img_paths = list(Path('/kaggle/input/rsna2025-series-yolo-prediction-plus-3d/series').glob('*/*/*/*/*'))
df = pd.DataFrame({'file_name': img_paths})
df

,file_name
0,/kaggle/input/rsna2025-series-yolo-prediction-...
1,/kaggle/input/rsna2025-series-yolo-prediction-...
2,/kaggle/input/rsna2025-series-yolo-prediction-...
3,/kaggle/input/rsna2025-series-yolo-prediction-...
4,/kaggle/input/rsna2025-series-yolo-prediction-...
...,...
8691,/kaggle/input/rsna2025-series-yolo-prediction-...
8692,/kaggle/input/rsna2025-series-yolo-prediction-...
8693,/kaggle/input/rsna2025-series-yolo-prediction-...
8694,/kaggle/input/rsna2025-series-yolo-prediction-...


In [6]:
model_paths = [f'/kaggle/input/rsna-yolo-models/yolo_cv_rot90/runs/fold{i}/weights/best.pt' for i in range(5)]
models = []
for mp in model_paths:
    model = YOLO(mp)
    models.append(model)

In [7]:
paths_s = df['file_name'].to_list()[:5]

In [ ]:
 results = ensemble_infer_dir(
    models,
    image_paths=df['file_name'].to_list(),
    out_dir="/kaggle/working/ens_out",
    conf=0.2,
    iou=0.7,
    imgsz=320,
    per_model_max_det=1,
    ensemble_iou=0.55,
    only_one=True,
    draw_score=True,
)

  4%|▍         | 362/8696 [01:10<25:21,  5.48it/s]

In [9]:
results

{'csv': '/kaggle/working/ens_out/ensemble_predictions.csv',
 'overlays_dir': '/kaggle/working/ens_out/overlays',
 'df':                                           image_path         x1          y1  \
 0  /kaggle/input/rsna2025-series-yolo-prediction-...  99.452400   42.049290   
 1  /kaggle/input/rsna2025-series-yolo-prediction-...  97.629387  103.363831   
 2  /kaggle/input/rsna2025-series-yolo-prediction-...  62.526955    0.859140   
 3  /kaggle/input/rsna2025-series-yolo-prediction-...  70.889900    0.022064   
 4  /kaggle/input/rsna2025-series-yolo-prediction-...  76.857033    1.796409   
 
            x2          y2     score  class          cx          cy          w  \
 0  171.198425  134.646469  0.999710      0  135.325413   88.347881  71.746025   
 1  155.986267  170.682159  0.998676      0  126.807827  137.022995  58.356880   
 2  138.272247   75.088318  0.999254      0  100.399601   37.973730  75.745293   
 3  153.113800   56.117504  0.995270      0  112.001850   28.069784  82

In [10]:
# df_plane = df[df['OrientationLabel']=='CORONAL']
# row = df_plane.iloc[4]
# img_files = row['sorted_files']
# sid = row['SeriesInstanceUID']
# modality = row['Modality']
# plane = row['OrientationLabel']
# if not isinstance(img_files, list):
#     img_file = list(Path(f'/kaggle/input/rsna-intracranial-aneurysm-detection/series/{sid}').glob('*'))[0]
#     imgs = load_dicom_array_3d(img_file)
# else:
#     imgs = load_dicom_array_2d(img_files)
#     y_spacing, x_spacing = row['PixelSpacing']
#     z_spacing = row['z_spacing']
#     imgs = resample_zyx(imgs,
#                         spacing=(z_spacing, y_spacing, x_spacing),
#                         new_spacing=(1., 1., 1.),
#                         order=1
#                        )
# if modality=='CTA':
#     imgs = value_normalize(imgs, -100, 600)
# else:
#     imgs = pct_normalize(imgs, 1, 99)